In [ ]:
import random
from firebase_admin import initialize_app, credentials, db

In [ ]:
cred = credentials.Certificate('creds.json')

In [ ]:
app = initialize_app(cred, {'databaseURL': "https://calzan-default-rtdb.europe-west1.firebasedatabase.app/"})

In [ ]:
h = "Holz"
l = "Lehm"
g = "Getreide"
w = "Wolle"
e = "Erz"
x = None

In [ ]:
seed = random.randrange(2 ** 32)
seed

In [ ]:
players = ['Simon', 'Dave', 'Heiko', 'Niko']
random.shuffle(players)

In [ ]:
    'tiles':  [
            h,  w,  w,
          g,  e,  g,  h,
        h,  l,  x,  e,  g,
          g,  e,  h,  w,
             l,  w,  l
    ],
    'rolls': [
            6,  3,  8,
          2,  4,  5, 10,
        5,  9,  0,  6,  9,
         10, 11,  3, 12,
            8,  4, 11
    ],


const tiles = [
    [-4, -4], [-4, 0], [-4, 4],
    [-2, -6], [-2, -2], [-2, 2], [-2, 6],
    [0, -8], [0, -4], [0, 0], [0, 4], [0, 8],
    [2, -6], [2, -2], [2, 2], [2, 6],
    [4, -4], [4, 0], [4, 4],
];

const oceans = [
    [-2, -10], //  0 left rising
    [-4,  -8], //  1 left rising
    [-6,  -6], //  2 left up
    [-6,  -2], //  3 up
    [-6,   2], //  4 up
    [-6,   6], //  5 right up
    [-4,   8], //  6 right falling
    [-2,  10], //  7 right falling
    [ 0,  12], //  8 right
    [ 2,  10], //  9 right rising
    [ 4,   8], // 10 right rising
    [ 6,   6], // 11 right down
    [ 6,  -2], // 12 down
    [ 6,   2], // 13 down
    [ 6,  -6], // 14 left down
    [ 4,  -8], // 15 left falling
    [ 2, -10], // 16 left falling
    [ 0, -12], // 17 left
];

const upNodes = tiles.concat(oceans.slice(9, 17)).map(([y, x]) => [y-1, x]);
const downNodes = tiles.concat(oceans.slice(0, 8)).map(([y, x]) => [y+1, x]);
const nodes = upNodes.concat(downNodes);

const risingEdges = tiles.concat(oceans.slice(9, 14)).map(([y, x]) => [y-1, x-1]);
const fallingEdges = tiles.concat(oceans.slice(3, 8)).map(([y, x]) => [y+1, x-1]);
const verticalEdges = tiles.concat(oceans.slice(6, 11)).map(([y, x]) => [y, x-2]);
const edges = risingEdges.concat(fallingEdges).concat(verticalEdges);





board = {}

for (var v = -5; v <= 5; v += 1) {
    for (var h = -10; h <= 10; h += 1) {
        const cell = {pos: [v, h]};
        state.board.push(cells);
    }
}
   
    upNodes.forEach((c) => getCell(c).node = 'up');
    downNodes.forEach((c) => getCell(c).node = 'down');
    verticalEdges.forEach((c) => getCell(c).edge = 'vert');
    risingEdges.forEach((c) => getCell(c).edge = 'rise');
    fallingEdges.forEach((c) => getCell(c).edge = 'fall');
    
    for (var tileNum = 0; tileNum < tiles.length; tileNum += 1) {
        const cell = getCell(tiles[tileNum]);
        if (cell.v == 0 && cell.h == 0) {
            cell.tile = W;
            cell.bandit = true;
        } else {
            cell.tile = data.tiles[tileNum];
            cell.roll = data.rolls[tileNum];
        }
    }

#ports = ['*', w, '*', e, '*', g, l, '*', h]

    board[-5][-5].port = '*';
    board[-5][-5].face = 'NW';
    board[-5][1].port = W;
    board[-5][1].face = 'NE';
    board[-3][7].port = '*';
    board[-3][7].face = 'NE';
    board[-2][-8].port = E;
    board[-2][-8].face = 'W';
    board[0][10].port = '*';
    board[0][10].face = 'E';
    board[2][-8].port = G;
    board[2][-8].face = 'W';
    board[3][7].port = L;
    board[3][7].face = 'SE';
    board[5][-5].port = '*';
    board[5][-5].face = 'SW';
    board[5][1].port = H;
    board[5][1].face = 'SE';
    */

In [ ]:
db.reference("test/initial").set({
    'seed': seed,
    'players': players,
    'board': board,
})

In [ ]:
import json

In [ ]:
with open("board.json") as json_file:
    board = json.load(json_file)
    db.reference("board").set(board)

In [ ]:
# shortcuts
              
def give(index, gains):
    
    resources = res_refs[index].get() or []
    resources += gains
    resources = sorted(resources, key=lambda r: resource_order[r])
    res_refs[index].set(resources)
    

def take(index, costs):
    
    resources = res_refs[index].get()
    for resource in costs:
        resources.remove(resource)
    res_refs[index].set(resources)
    

def draw():

    take(current_index, [g, w, e])
    
    stack = stack_ref.get()
    card_index = random.randrange(len(stack))
    card = stack.pop(card_index)
    stack_ref.set(stack)

    cards = card_refs[current_index].get() or []
    cards.append(card)
    card_refs[current_index].set(cards)
    
    
def bandit(row, col):

    board = board_ref.get()
    for r in range(row_count):
        for c in range(col_count):
            board[r][c]['bandit'] = r == row and c == col
    board_ref.set(board)
    
    for index, res_ref in enumerate(res_refs):
        resources = res_ref.get()
        res_count = len(resources)
        if len(resources) > 7:
            name = player_names[index]
            drop_count = res_count // 2
            print(f"{name} [{index}] must drop {drop_count} resources!")
    

def steal(from_index):

    resources = res_refs[from_index].get()
    steal_index = random.randrange(len(resources))
    resource = resources.pop(steal_index)
    res_refs[from_index].set(resources)
    
    give(current_index, [resource])
    
    
def knight(row, col, steal_index):
    
    cards = card_refs[current_index].get()
    cards.remove("Ritter")
    cards.append("Ritter (gespielt)")
    card_refs[current_index].set(cards)
    
    knight_count = cards.count("Ritter (gespielt)")
    turn = turn_ref.get()
    if knight_count > turn['maxKnightCount']:
        turn['maxKnightCount'] = knight_count
        turn['maxKnightIndex'] = current_index
        turn_ref.set(turn)
        
    bandit(row, col)
    steal(from_index)
    
    
def build(row1, col1, row2, col2):

    road(row1, col1)
    road(row2, col2)
    

def mono(res):

    gains = []
    
    for index, res_ref in enumerate(res_refs):
        if index == current_index:
            continue
        resources = res_ref.get()
        losses = [r for r in resources if r == res]
        take(index, losses)
        gains += losses
        
    give(current_index, gains)

        
def invent(res1, res2):
    
    give(current_index, [res1, res2])